In [672]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

In [673]:
# d = range(18202,18962)
match_range = range(18202,18242)

In [674]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [675]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

ConnectionError: HTTPSConnectionPool(host='understat.com', port=443): Max retries exceeded with url: /match/18226 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028C73E26310>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [738]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [739]:
all_res_data = backup_all_res_data

In [740]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [741]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [742]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [743]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [744]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
result_list = []
team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    result = []
    team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    xG_list.append(xG)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [745]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]

#Flattening nested lists

In [746]:
col_names = ['match_id','x','y','xG','minute','result','team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,result_list,team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
# df = pd.DataFrame([all_data[0],all_data[1],all_data[2],all_data[3],all_data[4],all_data[5],all_data[6],all_data[7],all_data[8],all_data[9],all_data[10]],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [747]:
df['away_score'] = 0
df['home_score'] = 0

Up next, need to figure out how to add score once we see goal, and keep it there for all of the subsequent rows for that game

In [748]:
df['goal_scored'] = ''

In [749]:
df['goal_scored'] = np.where((df['result'] == 'Goal') | (df['result'] == 'OwnGoal'), 1, 0)

In [750]:
away_count = 0

In [751]:
# df['home_score'] = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)
# df['away_score'] = np.where((df['result'] == 'Goal') & (df['h_a'] == 'a') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'h'), 1, 0)
# #Increment the score for each goal or own goal

In [752]:
df['neww'] = ''

In [753]:
for i in range(len(df)):
    yes = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 111, 0)

In [754]:
for i in range(len(df)):
    df['neww'][i+1] = yes[i]
    
#Setting the value one after the goal scored as equal to one, because technically at the time of the goalscoring shot, the 
#score would no 

C:\Users\User\AppData\Local\Temp/ipykernel_2712/862039262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['neww'][i+1] = yes[i]


In [755]:
df[20:42]

,index,match_id,x,y,xG,minute,result,team,h_a,player,home_score,away_score,goal_scored,neww
20,20,18202,0.9290000152587891,0.33,0.09606436640024185,86,BlockedShot,Arsenal,a,Bukayo Saka,0,0,0,0
21,21,18203,0.8859999847412109,0.664000015258789,0.06337210536003113,0,MissedShots,Fulham,h,Aleksandar Mitrovic,0,0,0,0
22,30,18203,0.9290000152587891,0.35900001525878905,0.016877776011824608,21,MissedShots,Liverpool,a,Mohamed Salah,0,0,0,0
23,22,18203,0.8530000305175781,0.44299999237060544,0.06387635320425034,23,BlockedShot,Fulham,h,João Palhinha,0,0,0,0
24,23,18203,0.8380000305175781,0.43900001525878907,0.06264717876911163,23,BlockedShot,Fulham,h,João Palhinha,0,0,0,0
25,31,18203,0.7780000305175782,0.49700000762939456,0.02267308346927166,28,BlockedShot,Liverpool,a,Jordan Henderson,0,0,0,0
26,24,18203,0.85,0.5029999923706054,0.07890216261148453,30,BlockedShot,Fulham,h,Harrison Reed,0,0,0,0
27,25,18203,0.9640000152587891,0.5670000076293945,0.08710415661334991,31,Goal,Fulham,h,Aleksandar Mitrovic,0,0,1,0
28,32,18203,0.9259999847412109,0.6869999694824219,0.061388105154037476,38,ShotOnPost,Liverpool,a,Luis Díaz,0,0,0,111
29,26,18203,0.8169999694824219,0.6609999847412109,0.04672102630138397,55,BlockedShot,Fulham,h,Andreas Pereira,0,0,0,0


In [ ]:
test_home_list = home_score_list

In [ ]:
test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [ ]:
test_home_list[20:42]

In [ ]:
test_home_list

In [ ]:
df